# Data Processing

## 0. Setup

In [1]:
# Automatic reloading
%load_ext autoreload
%autoreload 2

In [2]:
####################
# Required Modules #
####################

# Generic/Built-in
import sys
import os

# Libs
import pandas as pd

In [3]:
# Get the project directory 
current_dir = os.path.abspath('') # Current '\notebooks' directory
project_dir = os.path.abspath(os.path.join(current_dir, '..')) # Move up one level to project root directory

# Add the project directory to sys.path
sys.path.append(project_dir)

# Move up to project directory
os.chdir(project_dir)
os.getcwd()

'c:\\Users\\Ryan Lee\\Desktop\\50.038 Computational Data Science\\Digital-Asset-Prediction'

## 1. OHLC Data (Binance)

In [4]:
# Load OHLC data
ohlc = pd.read_csv("data/raw/binance_ohlc_data_20250327.csv")
ohlc['timestamp'] = pd.to_datetime(ohlc['timestamp'])
ohlc = ohlc.set_index('timestamp').sort_index()
ohlc.head()

,symbol,open,high,low,close,volume
timestamp,,,,,,
2022-03-24,BTC/USDT,42882.7600,44220.8900,42560.4600,43991.4600,5.619512e+04
2022-03-24,BNB/USDT,408.4000,416.6000,405.0000,414.2000,5.983661e+05
2022-03-24,CHR/USDT,0.4699,0.4906,0.4510,0.4756,4.414470e+07
2022-03-24,HIVE/USDT,0.9443,1.0248,0.9396,1.0248,4.736850e+06
2022-03-24,CTSI/USDT,0.3824,0.3915,0.3724,0.3890,1.173377e+07


## 2. Volume, Market Cap, Daily Return (Binance)

In [5]:
crypto = pd.read_csv("data/raw/binance_crypto_data_20250327.csv")
rows_with_na = crypto[crypto.isna().any(axis=1)]
print("Missing values:", len(rows_with_na))
rows_with_na

Missing values: 100


,date,price,volume,market_cap,daily_return,asset
0,2022-03-24,43991.46000,5.619512e+04,2.472106e+09,NaN,BTC/USDT
1097,2022-03-24,3110.76000,5.771715e+05,1.795442e+09,NaN,ETH/USDT
2194,2022-03-24,414.20000,5.983661e+05,2.478432e+08,NaN,BNB/USDT
3291,2022-03-24,24.40000,5.118541e+05,1.248924e+07,NaN,NEO/USDT
4388,2022-03-24,126.50000,4.946735e+05,6.257620e+07,NaN,LTC/USDT
...,...,...,...,...,...,...
103422,2022-03-24,29.71000,1.887342e+04,5.607293e+05,NaN,NMR/USDT
104519,2022-03-24,21.26000,7.157668e+06,1.521720e+08,NaN,DOT/USDT
105616,2022-03-24,93.64000,2.869856e+06,2.687333e+08,NaN,LUNA/USDT
106696,2022-03-24,0.01467,5.972397e+08,8.761507e+06,NaN,RSR/USDT


Each crypto is missing `market_cap` value for the first date (2022-03-24). Because the value for the first date is missing, we need to **backfill** the missing values.

In [6]:
crypto['date'] = pd.to_datetime(crypto['date'])
crypto = crypto.bfill() # Backfill missing market_cap values
crypto = crypto.set_index('date').sort_index()
crypto.head()

,price,volume,market_cap,daily_return,asset
date,,,,,
2022-03-24,43991.4600,5.619512e+04,2.472106e+09,0.007313,BTC/USDT
2022-03-24,414.2000,5.983661e+05,2.478432e+08,-0.009416,BNB/USDT
2022-03-24,0.4756,4.414470e+07,2.099522e+07,-0.038267,CHR/USDT
2022-03-24,1.0248,4.736850e+06,4.854324e+06,-0.021370,HIVE/USDT
2022-03-24,0.3890,1.173377e+07,4.564435e+06,-0.064267,CTSI/USDT


## 3. S&P500

In [7]:
sp500 = pd.read_csv("data/raw/sp500.csv")
rows_with_na = sp500[sp500.isna().any(axis=1)]
print("Missing values:", len(rows_with_na))
rows_with_na.head()

Missing values: 31


,realtime_start,realtime_end,date,value
16,2025-03-27,2025-03-27,2022-04-15,NaN
47,2025-03-27,2025-03-27,2022-05-30,NaN
62,2025-03-27,2025-03-27,2022-06-20,NaN
72,2025-03-27,2025-03-27,2022-07-04,NaN
117,2025-03-27,2025-03-27,2022-09-05,NaN


In [8]:
sp500['date'] = pd.to_datetime(sp500['date'])
sp500 = sp500.ffill() # Forward fill missing values
sp500 = sp500.set_index('date')['value'].sort_index() # Keep only 'value' column
sp500.head()

date
2022-03-24    4520.16
2022-03-25    4543.06
2022-03-28    4575.52
2022-03-29    4631.60
2022-03-30    4602.45
Name: value, dtype: float64

In [9]:
sp500

date
2022-03-24    4520.16
2022-03-25    4543.06
2022-03-28    4575.52
2022-03-29    4631.60
2022-03-30    4602.45
               ...   
2025-03-18    5614.66
2025-03-19    5675.29
2025-03-20    5662.89
2025-03-21    5667.56
2025-03-24    5767.57
Name: value, Length: 783, dtype: float64

## 4. Treasury Constant Maturity Spread

In [10]:
treasury = pd.read_csv("data/raw/treasury_constant_maturity_spread.csv")
rows_with_na = treasury[treasury.isna().any(axis=1)]
print("Missing values:", len(rows_with_na))
rows_with_na.head()

Missing values: 34


,realtime_start,realtime_end,date,value
16,2025-03-25,2025-03-25,2022-04-15,NaN
47,2025-03-25,2025-03-25,2022-05-30,NaN
62,2025-03-25,2025-03-25,2022-06-20,NaN
72,2025-03-25,2025-03-25,2022-07-04,NaN
117,2025-03-25,2025-03-25,2022-09-05,NaN


In [11]:
treasury['date'] = pd.to_datetime(treasury['date'])
treasury = treasury.ffill()
treasury = treasury.set_index('date')['value'].sort_index() # Keep only 'value' column
treasury.head()

date
2022-03-24    0.21
2022-03-25    0.18
2022-03-28    0.11
2022-03-29    0.06
2022-03-30    0.04
Name: value, dtype: float64

## 5. Fear & Greed Index

In [12]:
fear_greed = pd.read_csv("data/raw/fear_greed_index.csv")

fear_greed = fear_greed.drop(columns=["time_until_update"]) # unnecessary column
fear_greed["date"] = pd.to_datetime(fear_greed["date"])
fear_greed = fear_greed.set_index("date").sort_index()
fear_greed

,index_value,sentiment
date,,
2022-03-24,40,Fear
2022-03-25,47,Neutral
2022-03-26,51,Neutral
2022-03-27,49,Neutral
2022-03-28,60,Greed
...,...,...
2025-03-21,31,Fear
2025-03-22,32,Fear
2025-03-23,30,Fear


## 6. Gold Prices

In [13]:
gold = pd.read_csv("data/raw/Gold_price_averages_monthly.csv")
gold["date"] = pd.to_datetime(gold["date"], dayfirst=True)
gold = gold.set_index("date").sort_index()
gold = gold["USD"] # Looking at USD is sufficient (also no missing values)
gold

C:\Users\Ryan Lee\AppData\Local\Temp\ipykernel_14524\963842659.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gold["date"] = pd.to_datetime(gold["date"], dayfirst=True)


date
1978-01-31      207.8
1978-02-28      227.3
1978-03-31      245.7
1978-04-28      242.1
1978-05-31      239.2
               ...   
2024-10-31    2,690.1
2024-11-29    2,650.7
2024-12-31    2,644.1
2025-01-31    2,709.7
2025-02-28    2,894.7
Name: USD, Length: 566, dtype: object

In [14]:
# USD column being read as object as dataset contains some values wrapped in quotation marks
# e.g. "1,279.0"
gold.dtype

dtype('O')

In [15]:
# Clean strings (remove quotation marks) and convert
gold = gold.str.replace('"', '').str.replace(",", "").astype(float)
gold.dtype

dtype('float64')

## X. Combine
WIP

S&P500 and Treasury Constant Maturity Spread data do not have entries for certain dates. We will first align the data to the OHLC dates, and then forward-fill values for those missing dates.

In [16]:
# Align data to OHLC dates and forward-fill
sp500_filled = sp500.reindex(ohlc.index).ffill()
treasury_filled = treasury.reindex(ohlc.index).ffill()
fear_greed_filled = fear_greed.reindex(ohlc.index).ffill()

Additional processing needed for gold price feature: need to forward fill from cut off data

In [17]:
# Get the start date of your OHLC data
ohlc_start_date = ohlc.index.min()

# Find the last gold price BEFORE ohlc_start_date
last_gold_before_ohlc = gold[gold.index < ohlc_start_date].iloc[-1]  # Last row

# Add this value to gold data (to ensure forward-fill catches it)
gold_prefilled = gold.copy()
gold_prefilled.loc[ohlc_start_date - pd.Timedelta(days=1)] = last_gold_before_ohlc  # Add 1 day before OHLC starts
gold_prefilled = gold_prefilled.sort_index()

# Reindex to OHLC dates and forward-fill (now includes pre-start value)
gold_filled = gold_prefilled.reindex(ohlc.index, method="ffill")

In [18]:
gold_filled.head(703)

timestamp
2022-03-24    1856.3
2022-03-24    1856.3
2022-03-24    1856.3
2022-03-24    1856.3
2022-03-24    1856.3
               ...  
2022-03-30    1856.3
2022-03-30    1856.3
2022-03-31    1947.8
2022-03-31    1947.8
2022-03-31    1947.8
Name: USD, Length: 703, dtype: float64

In [19]:
combined = pd.concat([
    ohlc,
    crypto[['market_cap', 'daily_return']],
    sp500_filled.rename('sp500'),
    treasury_filled.rename('treasury_spread'),
    fear_greed_filled[['index_value']].rename(columns={'index_value': 'fear_greed'}),
    gold_filled.rename('gold_price_usd')
], axis=1)

combined.index.name = 'date'
combined.head()

,symbol,open,high,low,close,volume,market_cap,daily_return,sp500,treasury_spread,fear_greed,gold_price_usd
date,,,,,,,,,,,,
2022-03-24,BTC/USDT,42882.7600,44220.8900,42560.4600,43991.4600,5.619512e+04,2.472106e+09,0.007313,4520.16,0.21,40.0,1856.3
2022-03-24,BNB/USDT,408.4000,416.6000,405.0000,414.2000,5.983661e+05,2.478432e+08,-0.009416,4520.16,0.21,40.0,1856.3
2022-03-24,CHR/USDT,0.4699,0.4906,0.4510,0.4756,4.414470e+07,2.099522e+07,-0.038267,4520.16,0.21,40.0,1856.3
2022-03-24,HIVE/USDT,0.9443,1.0248,0.9396,1.0248,4.736850e+06,4.854324e+06,-0.021370,4520.16,0.21,40.0,1856.3
2022-03-24,CTSI/USDT,0.3824,0.3915,0.3724,0.3890,1.173377e+07,4.564435e+06,-0.064267,4520.16,0.21,40.0,1856.3


In [20]:
# Optional step: reorganize dataset (sort by symbol, then date index)
combined = combined.iloc[combined.reset_index().sort_values(by=["symbol", "date"]).index]
combined.head()

,symbol,open,high,low,close,volume,market_cap,daily_return,sp500,treasury_spread,fear_greed,gold_price_usd
date,,,,,,,,,,,,
2022-03-24,ADA/USDT,1.108,1.190,1.076,1.131,390982460.0,4.422012e+08,-0.030946,4520.16,0.21,40.0,1856.3
2022-03-25,ADA/USDT,1.130,1.167,1.072,1.096,290456883.7,3.183407e+08,-0.030946,4543.06,0.18,47.0,1856.3
2022-03-26,ADA/USDT,1.096,1.160,1.079,1.152,143117986.1,1.648719e+08,0.051095,4543.06,0.18,51.0,1856.3
2022-03-27,ADA/USDT,1.152,1.186,1.104,1.184,135043361.8,1.598913e+08,0.027778,4543.06,0.18,49.0,1856.3
2022-03-28,ADA/USDT,1.184,1.246,1.157,1.170,261105698.6,3.054937e+08,-0.011824,4575.52,0.11,60.0,1856.3


In [21]:
# Save combined dataset
combined.to_csv("data/processed/combined_dataset_v1.csv")